# LSTM Model 6

In [1]:
import pickle
X_train = pickle.load(open( 'feat_table/xgb_1_x_train_table.pkl', "rb" ))
y_train = pickle.load(open( 'feat_table/xgb_1_y_train_table.pkl', "rb" ))
X_valid = pickle.load(open( 'feat_table/xgb_1_x_valid_table.pkl', "rb" ))
y_valid = pickle.load(open( 'feat_table/xgb_1_y_valid_table.pkl', "rb" ))
test = pickle.load(open( 'feat_table/xgb_1_test_table.pkl', "rb" ))
features = pickle.load(open( 'feat_table/xgb_1_features_vector_vector.pkl', "rb" ))

In [8]:
# keras libs
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, Input
from keras.layers.recurrent import LSTM
import numpy as np
from keras import backend as K

In [4]:
# reformatting
x_train_array = np.copy(X_train[features].values)
x_valid_array = np.copy(X_valid[features].values)

format_sub_train_x = x_train_array.reshape((x_train_array.shape[0], 1, x_train_array.shape[1]) )
format_sub_val_x = x_valid_array.reshape((x_valid_array.shape[0], 1, x_valid_array.shape[1]) )

# custom loss function
def mean_squared_percentage_error(y_true, y_pred):
    return K.mean(K.square((y_true - y_pred) / K.clip(K.abs(y_true),
                                            K.epsilon(),
                                            None)), axis=-1)

In [10]:
format_sub_train_x.shape

(834205, 1, 16)

In [23]:
model_input = Input( shape = (format_sub_train_x.shape[1], format_sub_train_x.shape[2]) )
lstm_1 = LSTM(100, return_sequences=True)(model_input)
dropout_mod = Dropout(0.2)(lstm_1)
lstm_2 = LSTM(100)(dropout_mod)
dropout_mod_2 = Dropout(0.2)(lstm_2)
output = Dense(1)(dropout_mod_2)
model = Model(inputs = model_input, outputs = output)

In [24]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 1, 16)             0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 1, 100)            46800     
_________________________________________________________________
dropout_12 (Dropout)         (None, 1, 100)            0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 127,301
Trainable params: 127,301
Non-trainable params: 0
_________________________________________________________________
None

In [25]:
model.compile(loss=mean_squared_percentage_error, optimizer='adam')

In [26]:
mod_6 = model.fit(format_sub_train_x, np.copy(y_train), epochs=100, batch_size=100, 
                    validation_data=(format_sub_val_x, np.copy(y_valid)), verbose=2, shuffle=False)

Train on 834205 samples, validate on 10133 samples
Epoch 1/100
 - 44s - loss: 0.0098 - val_loss: 0.0026
Epoch 2/100
 - 39s - loss: 0.0040 - val_loss: 0.0024
Epoch 3/100
 - 39s - loss: 0.0033 - val_loss: 0.0025
Epoch 4/100
 - 39s - loss: 0.0028 - val_loss: 0.0024
Epoch 5/100
 - 39s - loss: 0.0026 - val_loss: 0.0024
Epoch 6/100
 - 39s - loss: 0.0025 - val_loss: 0.0024
Epoch 7/100
 - 39s - loss: 0.0024 - val_loss: 0.0024
Epoch 8/100
 - 39s - loss: 0.0024 - val_loss: 0.0024
Epoch 9/100
 - 39s - loss: 0.0024 - val_loss: 0.0024
Epoch 10/100
 - 39s - loss: 0.0024 - val_loss: 0.0023
Epoch 11/100
 - 39s - loss: 0.0023 - val_loss: 0.0022
Epoch 12/100
 - 39s - loss: 0.0023 - val_loss: 0.0022
Epoch 13/100
 - 39s - loss: 0.0023 - val_loss: 0.0024
Epoch 14/100
 - 39s - loss: 0.0023 - val_loss: 0.0023
Epoch 15/100
 - 39s - loss: 0.0023 - val_loss: 0.0022
Epoch 16/100
 - 39s - loss: 0.0023 - val_loss: 0.0022
Epoch 17/100
 - 39s - loss: 0.0023 - val_loss: 0.0022
Epoch 18/100
 - 39s - loss: 0.0023 - val

In [28]:
model.save('model/model_6.h5')